# Train Models

In [7]:
import numpy as np

import os
from pathlib import Path

from nvita.utils import open_json
from nvita.models.data import SplittedTSData
from nvita.models.cnn import CNN
from nvita.models.lstm import LSTM
from nvita.models.gru import GRU
from nvita.models.rf import RF
from nvita.models.utils import save_model, load_model
import nvita.models.train as mt

In [3]:
PATH_ROOT = Path(os.getcwd()).parent.absolute()

my_metadata = open_json(os.path.join(
    PATH_ROOT, "experiments", "metadata.json"))

model_paras = open_json(os.path.join(
    PATH_ROOT, "experiments", "model_paras.json"))
print(model_paras)

{'CNN': {'Electricity': [500, 0.005, 4, 256, 3, 256, 128, 1], 'NZTemp': [500, 0.01, 3, 64, 9, 64, 64, 1], 'CNYExch': [500, 0.005, 7, 256, 5, 256, 128, 1], 'Oil': [500, 0.01, 7, 256, 6, 256, 32, 1]}, 'LSTM': {'Electricity': [500, 0.01, 3, 64, 1, 1], 'NZTemp': [500, 0.01, 9, 256, 1, 1], 'CNYExch': [500, 0.01, 5, 64, 2, 1], 'Oil': [500, 0.01, 6, 128, 1, 1]}, 'GRU': {'Electricity': [500, 0.005, 3, 256, 1, 1], 'NZTemp': [500, 0.005, 9, 256, 4, 1], 'CNYExch': [500, 0.01, 5, 64, 1, 1], 'Oil': [500, 0.01, 6, 128, 4, 1]}, 'RF': {'Electricity': [500], 'NZTemp': [250], 'CNYExch': [100], 'Oil': [100]}}


In [4]:
SEEDS = my_metadata["seeds"]
MODELS = my_metadata["models"]
DATA_NAMES =  my_metadata["data"]

In [5]:
def train_models_based_on_model_paras(model_paras_dict, s_d):
    for model_name in MODELS:
        if model_name != "RF":
            if model_name == "CNN":
                epoch, lr, window_size, c, f0, f1, f2, out = model_paras_dict[model_name][s_d.df_name]
                model = CNN(window_size, c, f0, f1, f2, out)
            elif model_name == "LSTM":
                epoch, lr, input_dim, hidden_dim, num_layers, output_dim = model_paras_dict[model_name][s_d.df_name]
                model = LSTM(input_dim, hidden_dim, num_layers, output_dim)
            elif model_name == "GRU":
                epoch, lr, input_dim, hidden_dim, num_layers, output_dim = model_paras_dict[model_name][s_d.df_name]
                model = GRU(input_dim, hidden_dim, num_layers, output_dim)
            mt.train(model, lr, epoch, s_d.X_train, s_d.y_train)
        else:
            # model name is "RF"
            n_estimators = model_paras_dict[model_name][s_d.df_name][0]
            model = RF(n_estimators)
            model.fit(s_d.X_train, s_d.y_train)
        score = np.mean(mt.evaluate(model, s_d.X_test, s_d.y_test)**2)
        save_model(model, PATH_ROOT, s_d.df_name, s_d.seed)
        print("Best Score for " + model_name + " is " + str(score) + " on dataset " + str(s_d.df_name) + " with seed " + str(s_d.seed))
    

In [6]:
for data_ind in range(len(DATA_NAMES)):
    for seed in SEEDS:
        s_data = SplittedTSData()
        s_data = s_data.load_splitted_data(PATH_ROOT, DATA_NAMES[data_ind], seed)
        train_models_based_on_model_paras(model_paras, s_data)

Best Score for CNN is 0.015271303 on dataset Electricity with seed 2210
Best Score for LSTM is 0.012170031 on dataset Electricity with seed 2210
Best Score for GRU is 0.011077312 on dataset Electricity with seed 2210
Best Score for RF is 0.010321875968342518 on dataset Electricity with seed 2210
Best Score for CNN is 0.024276532 on dataset Electricity with seed 9999
Best Score for LSTM is 0.013433768 on dataset Electricity with seed 9999
Best Score for GRU is 0.0130042 on dataset Electricity with seed 9999
Best Score for RF is 0.012262301565136833 on dataset Electricity with seed 9999
Best Score for CNN is 0.040827297 on dataset Electricity with seed 58361
Best Score for LSTM is 0.00961763 on dataset Electricity with seed 58361
Best Score for GRU is 0.008230432 on dataset Electricity with seed 58361
Best Score for RF is 0.007801271029363551 on dataset Electricity with seed 58361
Best Score for CNN is 0.015714332 on dataset Electricity with seed 789789
Best Score for LSTM is 0.015916431